In [1]:
%%configure -f
{
    "conf": {
        "spark.jars": "https://repo1.maven.org/maven2/com/amazon/deequ/deequ/1.0.2/deequ-1.0.2.jar"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
12,application_1578964032290_0012,spark,idle,Link,Link,


In [2]:
import com.amazon.deequ.VerificationSuite
import com.amazon.deequ.analyzers.Size
import com.amazon.deequ.anomalydetection.RelativeRateOfChangeStrategy
import com.amazon.deequ.repository.ResultKey
import com.amazon.deequ.repository.memory.InMemoryMetricsRepository
import com.amazon.deequ.checks.CheckStatus._
import org.apache.spark.sql.{DataFrame, SparkSession}

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
13,application_1578964032290_0013,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import com.amazon.deequ.VerificationSuite
import com.amazon.deequ.analyzers.Size
import com.amazon.deequ.anomalydetection.RelativeRateOfChangeStrategy
import com.amazon.deequ.repository.ResultKey
import com.amazon.deequ.repository.memory.InMemoryMetricsRepository
import com.amazon.deequ.checks.CheckStatus._
import org.apache.spark.sql.{DataFrame, SparkSession}


In [4]:
val session = SparkSession.builder().master("local").appName("test").config("spark.ui.enabled", "false").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

session: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5decd65c


In [6]:
case class Item(
    id: Long,
    productName: String,
    description: String,
    priority: String,
    numViews: Long
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined class Item


In [7]:
  def itemsAsDataframe(session: SparkSession, items: Item*): DataFrame = {
    val rdd = session.sparkContext.parallelize(items)
    session.createDataFrame(rdd)
  }

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

itemsAsDataframe: (session: org.apache.spark.sql.SparkSession, items: Item*)org.apache.spark.sql.DataFrame


In [8]:
   /* Anomaly detection operates on metrics stored in a metric repository, so lets create one */
    val metricsRepository = new InMemoryMetricsRepository()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

metricsRepository: com.amazon.deequ.repository.memory.InMemoryMetricsRepository = com.amazon.deequ.repository.memory.InMemoryMetricsRepository@9bd634b


In [9]:
    /* This is the key which we use to store the metrics for the dataset from yesterday */
    val yesterdaysKey = ResultKey(System.currentTimeMillis() - 24 * 60 * 1000)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

yesterdaysKey: com.amazon.deequ.repository.ResultKey = ResultKey(1579490603123,Map())


In [10]:
    /* Yesterday, the data had only two rows */
    val yesterdaysDataset = itemsAsDataframe(session,
      Item(1, "Thingy A", "awesome thing.", "high", 0),
      Item(2, "Thingy B", "available at http://thingb.com", null, 0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

yesterdaysDataset: org.apache.spark.sql.DataFrame = [id: bigint, productName: string ... 3 more fields]


In [13]:
/* We test for anomalies in the size of the data, it should not increase by more than 2x. Note
    that we store the resulting metrics in our repository */
VerificationSuite().onData(yesterdaysDataset).useRepository(metricsRepository).saveOrAppendResult(yesterdaysKey).addAnomalyCheck(RelativeRateOfChangeStrategy(maxRateIncrease = Some(2.0)), Size()).run()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res9: com.amazon.deequ.VerificationResult = VerificationResult(Warning,Map(Check(Warning,Anomaly check for Size(None),List(AnomalyConstraint(Size(None)))) -> CheckResult(Check(Warning,Anomaly check for Size(None),List(AnomalyConstraint(Size(None)))),Warning,List(ConstraintResult(AnomalyConstraint(Size(None)),Failure,Some(Can't execute the assertion: requirement failed: There have to be previous results in the MetricsRepository!!),Some(DoubleMetric(Dataset,Size,*,Success(2.0))))))),Map(Size(None) -> DoubleMetric(Dataset,Size,*,Success(2.0))))


In [14]:
    /* Todays data has five rows, so the data size more than doubled and our anomaly check should
       catch this */
    val todaysDataset = itemsAsDataframe(session,
      Item(1, "Thingy A", "awesome thing.", "high", 0),
      Item(2, "Thingy B", "available at http://thingb.com", null, 0),
      Item(3, null, null, "low", 5),
      Item(4, "Thingy D", "checkout https://thingd.ca", "low", 10),
      Item(5, "Thingy E", null, "high", 12))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

todaysDataset: org.apache.spark.sql.DataFrame = [id: bigint, productName: string ... 3 more fields]


In [15]:
    /* The key for today's result */
    val todaysKey = ResultKey(System.currentTimeMillis())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

todaysKey: com.amazon.deequ.repository.ResultKey = ResultKey(1579492168395,Map())


In [16]:
    /* Repeat the anomaly check for today's data */
    val verificationResult = VerificationSuite().onData(todaysDataset).useRepository(metricsRepository).saveOrAppendResult(todaysKey).addAnomalyCheck(RelativeRateOfChangeStrategy(maxRateIncrease = Some(2.0)), Size()).run()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

verificationResult: com.amazon.deequ.VerificationResult = VerificationResult(Warning,Map(Check(Warning,Anomaly check for Size(None),List(AnomalyConstraint(Size(None)))) -> CheckResult(Check(Warning,Anomaly check for Size(None),List(AnomalyConstraint(Size(None)))),Warning,List(ConstraintResult(AnomalyConstraint(Size(None)),Failure,Some(Value: 5.0 does not meet the constraint requirement!),Some(DoubleMetric(Dataset,Size,*,Success(5.0))))))),Map(Size(None) -> DoubleMetric(Dataset,Size,*,Success(5.0))))


In [17]:
    /* Did we find an anomaly? */
    if (verificationResult.status != Success) {
      println("Anomaly detected in the Size() metric!")

      /* Lets have a look at the actual metrics. */
      metricsRepository
        .load()
        .forAnalyzers(Seq(Size()))
        .getSuccessMetricsAsDataFrame(session)
        .show()
    }

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Anomaly detected in the Size() metric!
+-------+--------+----+-----+-------------+
| entity|instance|name|value| dataset_date|
+-------+--------+----+-----+-------------+
|Dataset|       *|Size|  2.0|1579490603123|
|Dataset|       *|Size|  5.0|1579492168395|
+-------+--------+----+-----+-------------+

